In [1]:
import numpy as np
import pandas as pd
from PIL import Image
from datasets import Dataset, Features, Image, load_dataset

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision.transforms import v2

In [2]:
# Hyper-parameters
learning_rate = 1e-3
batch_size = 64

In [3]:
# Import data
dataset = load_dataset("camfruss/bread_proofing")

In [4]:
train_df = pd.DataFrame(dataset["train"])
valid_df = pd.DataFrame(dataset["valid"])
test_df = pd.DataFrame(dataset["test"])

In [41]:
# https://huggingface.co/transformers/v3.2.0/custom_datasets.html
class BreadProofingDataset(torch.utils.data.Dataset):
    
    def __init__(self, df):
        self.df = df

    def __getitem__(self, idx):
        image = np.array(self.df.iloc[idx]["image"].convert("RGB"), dtype=np.float32)
        image = np.einsum("ijk->kij", image)
        image = torch.from_numpy(image)
        
        row = self.df.iloc[idx]
        label_cols = ["under_proof", "over_proof", "perfect_proof", "unsure_proof"]
        label = torch.tensor(row[label_cols].astype(np.float32), dtype=torch.float, requires_grad=True)
        
        return image, label

    def __len__(self):
        return len(self.df)

In [43]:
def create_dl(df):
    return DataLoader(BreadProofingDataset(df), batch_size=batch_size)

train_dl = create_dl(train_df)
valid_dl = create_dl(valid_df)
test_dl = create_dl(test_df)

In [44]:
dim = 2**7
out_channels = 16
fc_features = out_channels * dim // 4

class ConvolutionalNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn = nn.Sequential(
            v2.Grayscale(),
            v2.Resize((dim, dim)),
            
            nn.Conv2d(in_channels=1, out_channels=8, kernel_size=5, stride=1),  # default padding of 0s
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # halves
            
            nn.Conv2d(in_channels=8, out_channels=out_channels, kernel_size=5, stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # halves
            
            nn.Flatten(),
            
            nn.Linear(in_features=fc_features, out_features=fc_features // 4),
            nn.ReLU(),
            
            nn.Linear(in_features=fc_features // 4, out_features=4),
            nn.Softmax()
        )

    def forward(self, x):
        return self.cnn(x)

In [45]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

model = ConvolutionalNN().to(device)

Using cpu device


In [46]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [47]:
def train(dataloader):
    model.train()
    
    size = len(dataloader.dataset)
    for idx, (X, y) in enumerate(dataloader):
        optimizer.zero_grad()
    
        pred = model(X)
        loss = loss_fn(pred, y)
        loss.backward()
        optimizer.step()
        
        if idx % 100 == 0:
            loss, current = loss.item(), idx * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [48]:
def test(dataloader, split="Validation"):
    model.eval()
    
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"{split} Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")     

In [49]:
epochs = 10

for t in range(epochs):
    print(f"Epoch {t+1}\n{20*'-'}")
    train(train_dl)
    test(valid_dl)

print(f"Completed {epochs} epochs!")

test(test_dl, split="Test")

Epoch 1
--------------------


/var/folders/ly/g4z0vf4n0rjb28ss4fhd81680000gn/T/ipykernel_8437/3678972042.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  label = torch.tensor(row[label_cols].astype(np.float32), dtype=torch.float, requires_grad=True)
/var/folders/ly/g4z0vf4n0rjb28ss4fhd81680000gn/T/ipykernel_8437/3678972042.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  label = torch.tensor(row[label_cols].astype(np.float32), dtype=torch.float, requires_grad=True)
/var/folders/ly/g4z0vf4n0rjb28ss4fhd81680000gn/T/ipykernel_8437/3678972042.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future versio

RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x13456 and 512x128)